In [9]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
print(torch.cuda.is_available(), torch.cuda.get_device_name())
device = torch.device(0)
print(device)

In [10]:
from huggingface_hub import login
login()

In [ ]:
# %pip install transformers datasets evaluate accelerate
model_id = "meta-llama/Meta-Llama-3-8B"
# model_id = 'google-bert/bert-base-uncased'
# translator = pipeline('machine-translation', model=model_id)
tg_bot= pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

In [ ]:
print(tg_bot('How is the weather?'))
print(tg_bot("Hey how are you doing today?"))
tg_bot("What is the US president's name?")

In [ ]:
quantization = BitsAndBytesConfig(load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True)

model = LlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", device_map="auto", quantization_config=quantization, offload_buffers=True)
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.pad_token = tokenizer.eos_token
prompt = ["Hey, are you conscious? Can you talk to me?", "What are the seven days of the week?", "Translate from English to French: 'Today is sunny.'"]
inputs = tokenizer(prompt, return_tensors="pt", padding=True).to('cuda')

# Generate
generated_ids = model.generate(inputs.input_ids)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(load_in_4bit=True)

gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
gemma_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", load_in_4bit=True)

In [ ]:
input_text = ["Write me a poem about Machine Learning.", "Who is the FIrst U.S. President?", "What is 2+2?", "Translate from English to French: It is a sunny day."]
input_ids = gemma_tokenizer(input_text, padding='longest', return_tensors="pt").to("cuda")

outputs = gemma_model.generate(**input_ids)
print([gemma_tokenizer.decode(out) for out in outputs])


In [ ]:
from transformers import AutoModelForCausalLM
mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

In [ ]:
input_text = ["Write me a poem about Machine Learning.", "Who is the FIrst U.S. President?", "What is 2+2?", "Translate from English to French: It is a sunny day."]
input_ids = mistral_tokenizer(input_text, padding='longest', return_tensors="pt").to("cuda")

outputs = mistral_model.generate(**input_ids)
print([mistral_tokenizer.decode(out) for out in outputs])


In [ ]:
tg_bot('What is 5 + 5')

In [ ]:
tg_bot('What is the capital city of Mexico')

In [11]:
model_id = "google-t5/t5-small"
translator = pipeline('translation_en_to_fr', model=model_id, device_map='auto')

In [12]:
prompts = ['my name is fat albert', 'he is fat and his name is albert', 'why is the dog so big and red?']
translator(prompts)

[{'translation_text': 'mon nom est fat albert'},
 {'translation_text': 'il est gras et son nom est albert'},
 {'translation_text': 'Pourquoi le chien est-il si gros et si rouge?'}]

In [22]:
sent_pipe = pipeline("sentiment-analysis", device=0)
sent_pipe('That was alright')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\abhiv\miniconda3\envs\mse244\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.999680757522583}]

In [27]:
from numba import cuda
device = cuda.get_current_device()
device.reset() 
# COULD ALSO USE -> torch.cuda.empty_cache() 

In [52]:
tg_pipe = pipeline("text-generation")
generated_output = tg_pipe('Generate an AP Physics C question:')

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [53]:
generated_output[0]['generated_text']

'Generate an AP Physics C question: Which model and answer should I use in this study? Here are some questions to keep in mind: How do I calculate if it is correct? Which assumptions do the C model and the answer to the test make'

In [57]:
tg_pipe("Give me a 30 word poem:")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Give me a 30 word poem:\n\n\nThe "goody bear"\n\n\nThe "tender head"\n\n\nThe "frosted brow," and the "distant spirit" -\n\n\nAnd I\'ll have a 30 word poem'